Inserción de los datos de Norman en la base de datos de ArangoDB

In [2]:
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_TYPE = "openai"

In [3]:
# Instantiate ArangoDB Database
import json

from adb_cloud_connector import get_temp_credentials
from arango import ArangoClient

con = get_temp_credentials()

db = ArangoClient(hosts=con["url"]).db(
    con["dbName"], con["username"], con["password"], verify=True
)

print(json.dumps(con, indent=2))

Success: reusing cached credentials
{
  "dbName": "TUTj7i7fm55rmp58xz4lw6skg",
  "username": "TUTbu0jwwiqud451fyooi8ww4",
  "password": "TUTue5dopxl90qrvij44492e",
  "hostname": "tutorials.arangodb.cloud",
  "port": 8529,
  "url": "https://tutorials.arangodb.cloud:8529"
}


In [4]:
# Instantiate the ArangoDB-LangChain Graph
from langchain_community.graphs import ArangoGraph
graph = ArangoGraph(db)

Se elimina el grafo Norman si ya existe y se crean las colecciones necesarias



In [5]:
if db.has_graph("Norman"):
    db.delete_graph("Norman", drop_collections=True)

db.create_graph(
    "Norman",
    edge_definitions=[
        {
            "edge_collection": "HasMedicalRecord",
            "from_vertex_collections": ["Persons"],
            "to_vertex_collections": ["MedicalRecords"],
        },
    ],
)

medical_records_01 = [
  {
    "_key": "record_2024_01_05",
    "datetime": "2024-01-05T10:20:00Z",
    "description": "Annual dental check-up and cleaning",
    "type": "check-up",
    "medical_specialty": "Dentistry",
    "name_of_doctor": "Dr. Isabel Torres",
    "symptoms_experienced": "None, routine visit",
    "hospital_or_facility": "Buenos Aires Dental Care",
    "notes": "No cavities, recommended wisdom tooth monitoring."
  },
  {
    "_key": "record_2024_01_12",
    "datetime": "2024-01-12T08:45:00Z",
    "description": "Blood pressure monitoring follow-up",
    "type": "check-up",
    "medical_specialty": "Cardiology",
    "name_of_doctor": "Dr. Luis Rodriguez",
    "symptoms_experienced": "",
    "hospital_or_facility": "",
    "results": [
      {
        "type": "Blood pressure reading",
        "value": ["120/80 mmHg"]
      }
    ],
    "description_of_study": "",
    "medications_prescribed": [],
    "therapy_received": [],
    "notes": ""
  },
]

edges = [
    {"_to": "Persons/Norman", "_from": "MedicalRecords/record_2024_01_05"},
    {"_to": "Persons/Norman", "_from": "MedicalRecords/record_2024_01_12"}
]

db.collection("MedicalRecords").import_bulk(medical_records_01)
db.collection("HasMedicalRecord").import_bulk(edges)

{'error': False,
 'created': 2,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

In [7]:
graph.set_schema()

In [8]:
# We can now view the generated schema

import json

print(json.dumps(graph.schema, indent=4))

{
    "Graph Schema": [
        {
            "graph_name": "Norman",
            "edge_definitions": [
                {
                    "edge_collection": "HasMedicalRecord",
                    "from_vertex_collections": [
                        "Persons"
                    ],
                    "to_vertex_collections": [
                        "MedicalRecords"
                    ]
                }
            ]
        }
    ],
    "Collection Schema": [
        {
            "collection_name": "MedicalRecords",
            "collection_type": "document",
            "document_properties": [
                {
                    "name": "_key",
                    "type": "str"
                },
                {
                    "name": "_id",
                    "type": "str"
                },
                {
                    "name": "_rev",
                    "type": "str"
                },
                {
                    "name": "datetime",
          

In [10]:
from langchain.chains import ArangoGraphQAChain
from langchain_openai import ChatOpenAI

chain = ArangoGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True, allow_dangerous_requests=True
)

Hacemos una pregunta desde un tercero ajeno a la persona de estudio en lenguaje natural

In [11]:
chain.run("cuantos hijos tiene normancito?")

/var/folders/g5/sts_zwf50495g151b7vd71840000gp/T/ipykernel_12493/103968715.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("cuantos hijos tiene normancito?")




> Entering new ArangoGraphQAChain chain...
AQL Query (1):
// This is not a valid question for generating an AQL query. Please provide a valid query related to the ArangoDB Schema.

AQL Query Execution Error: 
AQL: syntax error, unexpected end of query string near '' at position 3:1 (while parsing)

AQL Query (2): 
// This is not a valid question for generating an AQL query. Please provide a valid query related to the ArangoDB Schema.

AQL Query Execution Error: 
AQL: syntax error, unexpected end of query string near '' at position 3:1 (while parsing)

AQL Query (3):// This is not a valid question for generating an AQL query. Please provide a valid query related to the ArangoDB Schema.
AQL Query Execution Error: 
AQL: syntax error, unexpected end of query string near ' This is not a valid question fo...' at position 1:3 (while parsing)



ValueError: 
                Maximum amount of AQL Query Generation attempts reached.
                Unable to execute the AQL Query due to the following error:
                AQL: syntax error, unexpected end of query string near ' This is not a valid question fo...' at position 1:3 (while parsing)
            

In [24]:
chain.run("quien es el hijo de Norman?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, ChildOf
FOR v, e IN 1..1 OUTBOUND 'Persons/Norman' ChildOf
    RETURN v

AQL Result:
[{'_key': 'Carlos', '_id': 'Persons/Carlos', '_rev': '_huxYtma--D', 'name': 'Carlos', 'family_name': 'Gonzalez', 'age': 68, 'relationship_to_Norman': 'Father'}]

> Finished chain.


'Summary:\nThe son of Norman is Carlos. Carlos is 68 years old and has a father-son relationship with Norman.'

In [27]:
chain.run("add a new son of Norman named Xavier")





> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, ChildOf
LET norman = (
    FOR person IN Persons
    FILTER person.name == 'Norman'
    RETURN person
)[0]

LET xavier = {
    "_key": "Xavier",
    "name": "Xavier",
    "family_name": "Gonzalez",
    "age": 0
}

INSERT xavier INTO Persons

LET edge = {
    "_from": norman._id,
    "_to": CONCAT('Persons/', xavier._key)
}

INSERT edge INTO ChildOf

AQL Result:
[]

> Finished chain.


'A new son named Xavier has been successfully added to Norman in the database.'

In [28]:
chain.run("Norman tiene un hijo Xavier?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Persons, ChildOf
FOR v, e, p IN 1..1 OUTBOUND 'Persons/Norman' ChildOf
FILTER v.name == 'Xavier'
RETURN v

AQL Result:
[{'_key': 'Xavier', '_id': 'Persons/Xavier', '_rev': '_hux4B6W---', 'name': 'Xavier', 'family_name': 'Gonzalez', 'age': 0}]

> Finished chain.


"Summary:\nYes, Norman has a son named Xavier. According to the database query results, Xavier is related to Norman as his son, with the same family name 'Gonzalez'."